How to run this code:
Basically, we just follow the order of the cells(top to bottom)
(1) first import useful packages: levenshtein distance, nn modules, data augmentation packges

(2) second import necessary root and path to download the corresponding Kaggle data set.

(3) next, construct data loaders, and define the train, validation, and test dataloader

(4) build the attention model: encoder(plstm), decoder, attention calulation module

(5) train the base model(with tf = 1) unitl converge. Save the model, and keep training with decaying tf rate. Use defined functions to generate text(greedy) and calculate the levenshtein distance and do validation for each epoch.validate model by using Dev dataset(check validation loss and resulting distance)

(6) make prediction on test data with greedy search ->output to kaggle.

In [1]:
!pip install python-levenshtein

     |████████████████████████████████| 50 kB 2.9 MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149864 sha256=044596e831d40a34b66fabce159d5b2e8382a1be89db96b1cf19b9d3a397b8a9
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-levenshtein


# Libraries and Initial Processing

In [2]:
!pip install torch==1.11.0+cu113

In [3]:
import os
import sys
import pandas as pd
import numpy as np
import Levenshtein as lev
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.utils as utils
import seaborn as sns
import matplotlib.pyplot as plt
import time
import random
import datetime
from torch.utils import data
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm


cuda = torch.cuda.is_available()

print(cuda, sys.version)

device = torch.device("cuda" if cuda else "cpu")
num_workers = 4 if cuda else 0
print("Cuda = "+str(cuda)+" with num_workers = "+str(num_workers))
np.random.seed(11785)
torch.manual_seed(11785)

# The labels of the dataset contain letters in LETTER_LIST.
# You should use this to convert the letters to the corresponding indices
# and train your model with numerical labels.
LETTER_LIST = ['<sos>', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', \
         'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', "'", ' ', '<eos>']

True 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
Cuda = True with num_workers = 4


In [4]:
print(len(LETTER_LIST))

30


In [5]:
import torchaudio

"""train_audio_transforms = nn.Sequential(
    
    torchaudio.transforms.FrequencyMasking(freq_mask_param=3),
    torchaudio.transforms.TimeMasking(time_mask_param=200))"""
train_audio_transforms = nn.Sequential(
    torchaudio.transforms.FrequencyMasking(freq_mask_param=3),
    torchaudio.transforms.TimeMasking(time_mask_param=150)
)

In [6]:
def create_dictionaries(letter_list):
    '''
    Create dictionaries for letter2index and index2letter transformations
    based on LETTER_LIST

    Args:
        letter_list: LETTER_LIST

    Return:
        letter2index: Dictionary mapping from letters to indices
        index2letter: Dictionary mapping from indices to letters
    '''
    letter2index = dict()
    index2letter = dict()
    # TODO
    for idx, letter in enumerate(letter_list):
      letter2index[letter] = idx
      index2letter[idx] = letter
    return letter2index, index2letter
    
# used for generate sentence for a single batch
def transform_index_to_letter(batch_indices):
    '''
    Transforms numerical index input to string output by converting each index 
    to its corresponding letter from LETTER_LIST

    Args:
        batch_indices: List of indices from LETTER_LIST with the shape of (N, )
    
    Return:
        transcripts: List of converted string transcripts. This would be a list with a length of N
    '''
    transcripts = []
    for seq in batch_indices:
        sentence = ""
        for idx in seq:
            # Reached the end of the sentence
            if idx == letter2index['<eos>']:
                break
            elif idx == letter2index['<sos>']:
                continue
            else:
                sentence += index2letter[idx]
        transcripts.append(sentence)
    return transcripts
        
# Create the letter2index and index2letter dictionary
letter2index, index2letter = create_dictionaries(LETTER_LIST)

# Kaggle (TODO)

In [7]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"","key":""}') # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     |████████████████████████████████| 59 kB 3.0 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73275 sha256=e53884f235ba087168e419a6664113d10c1a52101526a63f490352c740576a14
  Stored in directory: /root/.cache/pip/wheels/de/f7/d8/c3902cacb7e62cb611b1ad343d7cc07f42f7eb76ae3a52f3d1
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [8]:
from IPython.core.display import JSON
!pip install --upgrade --force-reinstall --no-deps kaggle
#!mkdir /root/.kaggle
import json
token = {"username":"shaofengqin","key":"f8b852f6077e65972f63df09c96f42d3"}
! pwd
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(token,file)
for i in range(2):
  !chmod 600 /root/.kaggle/kaggle.json
  !cp /root/.kaggle/kaggle.json ~/.kaggle/
  !kaggle config set -n path -v /root
  !cat /root/.kaggle/kaggle.json

     |████████████████████████████████| 58 kB 3.1 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=8afe8b752b996fed18e05c0179112212ecc5bed73925f9506bf780b16156fbb5
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
/content
cp: '/root/.kaggle/kaggle.json' and '/root/.kaggle/kaggle.json' are the same file
- path is now set to: /root
{
  "username": "shaofengqin",
  "key": "f8b852f6077e65972f63df09c96f42d3",
  "path": "/root"
}cp: '/root/.kaggle/kaggle.json' and '/root/.kaggle/kaggle.json' are the same file
- path is now set to: /root
{
  "username": "shaofengqin",
  "key": "f8b852f6077e65972f63df09c96f42d3",
  "path": "/root"
}

In [9]:
!kaggle competitions download --force -c 11-785-s22-hw4p2



!unzip /root/competitions/11-785-s22-hw4p2/11-785-s22-hw4p2.zip


!ls

Streaming output truncated to the last 5000 lines.
  inflating: hw4p2_student_data/hw4p2_student_data/train/transcript/7113-086041-020.npy  
  inflating: hw4p2_student_data/hw4p2_student_data/train/transcript/7113-086041-021.npy  
  inflating: hw4p2_student_data/hw4p2_student_data/train/transcript/7113-086041-022.npy  
  inflating: hw4p2_student_data/hw4p2_student_data/train/transcript/7113-086041-023.npy  
  inflating: hw4p2_student_data/hw4p2_student_data/train/transcript/7113-086041-024.npy  
  inflating: hw4p2_student_data/hw4p2_student_data/train/transcript/7113-086041-025.npy  
  inflating: hw4p2_student_data/hw4p2_student_data/train/transcript/7113-086041-026.npy  
  inflating: hw4p2_student_data/hw4p2_student_data/train/transcript/7113-086041-027.npy  
  inflating: hw4p2_student_data/hw4p2_student_data/train/transcript/7113-086041-028.npy  
  inflating: hw4p2_student_data/hw4p2_student_data/train/transcript/7113-086041-029.npy  
  inflating: hw4p2_student_data/hw4p2_student_dat

# Dataset and Dataloading (TODO)

You will need to implement the Dataset class by your own. You can implement it similar to HW3P2. However, you are welcomed to do it your own way if it is more comfortable or efficient.

Note that you need to use LETTER_LIST to convert the transcript into numerical labels for the model.


Example of raw transcript:

    ['<sos>', 'N', 'O', 'R', 'T', 'H', 'A', 'N', 'G', 'E', 'R', ' ','A', 'B', 'B', 'E', 'Y', '<eos>']

Example of converted transcript ready to process for the model:

    [0, 14, 15, 18, 20, 8, 1, 14, 7, 5, 18, 28, 1, 2, 2, 5, 25, 29]


In [10]:
# dataset construction from HW3 Part 2
# This cell is where your actual TODOs start
# You will need to implement the Dataset class by your own. You may also implement it similar to HW1P2 (dont require context)
# The steps for implementation given below are how we have implemented it.
# However, you are welcomed to do it your own way if it is more comfortable or efficient. 

class LibriSamples(torch.utils.data.Dataset):

    def __init__(self, data_path, partition= "train"): # You can use partition to specify train or dev

        self.X_dir = data_path + "/" + partition + "/mfcc/" # TODO: get mfcc directory path
        self.Y_dir = data_path + "/" + partition +"/transcript/" # TODO: get transcript path

        self.X_files = os.listdir(self.X_dir)# TODO: list files in the mfcc directory
        self.Y_files = os.listdir(self.Y_dir)# TODO: list files in the transcript directory
        

        assert(len(self.X_files) == len(self.Y_files))

        pass

    def __len__(self):
        return len(self.X_files)

    def __getitem__(self, ind):
        X_path = self.X_dir + self.X_files[ind]
        Y_path = self.Y_dir + self.Y_files[ind]
        X = np.load(X_path)# TODO: Load the mfcc npy file at the specified index ind in the directory
        Y = np.load(Y_path) # TODO: Load the corresponding transcripts

        
        Yy = torch.LongTensor([letter2index[yy] for yy in Y])

        # X = torch.Tensor(x, dtype = )
        # X = troch.transpose(X) --pass to trnaform
        # data augmentation tranfrom on x

        return X, Yy
    
    def collate_fn(self,batch):

        batch_x = [torch.Tensor(x) for x,y in batch]
        batch_y = [torch.LongTensor(y) for x,y in batch]

        batch_x_pad = pad_sequence(batch_x, batch_first=True)# TODO: pad the sequence with pad_sequence (already imported)
        lengths_x = [len(X) for X in batch_x]# TODO: Get original lengths of the sequence before padding

        batch_y_pad = pad_sequence(batch_y, batch_first=True) # TODO: pad the sequence with pad_sequence (already imported)
        lengths_y = [len(Y) for Y in batch_y]# TODO: Get original lengths of the sequence before padding

        return batch_x_pad, batch_y_pad, torch.LongTensor(lengths_x), torch.LongTensor(lengths_y)


# You can either try to combine test data in the previous class or write a new Dataset class for test data
class LibriSamplesTest(torch.utils.data.Dataset):

    def __init__(self, data_path, test_order): # test_order is the csv similar to what you used in hw1
        partition= "test"
        test_order_path = data_path + "/" + partition + "/" + test_order

        test_order_list = list(pd.read_csv(test_order_path).file)# TODO: open test_order.csv as a list
        self.X = []
        for test_file in test_order_list:
          path = data_path + "/" + partition + "/mfcc/" + test_file
          (self.X).append(np.load(path))
        # TODO: Load the npy files from test_order.csv and append into a list
        # You can load the files here or save the paths here and load inside __getitem__ like the previous class
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, ind):
        # TODOs: Need to return only X because this is the test dataset
        return self.X[ind]
    
    def collate_fn(self,batch):
        batch_x = [torch.Tensor(x) for x in batch]
        batch_x_pad = pad_sequence(batch_x, batch_first=True)# TODO: pad the sequence with pad_sequence (already imported)
        lengths_x = [len(X) for X in batch_x]# TODO: Get original lengths of the sequence before padding

        return batch_x_pad, torch.LongTensor(lengths_x)

In [11]:
batch_size = 128
# I will put this in hyper_parameters-->easier to change

root = '/content/hw4p2_student_data/hw4p2_student_data'

train_data = LibriSamples(root, 'train')
val_data = LibriSamples(root, 'dev')
test_data = LibriSamplesTest(root, 'test_order.csv')

train_loader = DataLoader(train_data, shuffle=True, batch_size = batch_size, collate_fn = train_data.collate_fn,  pin_memory=True)# TODO: Define the train loader. Remember to pass in a parameter (function) for the collate_fn argument 
val_loader = DataLoader(val_data, shuffle= False, batch_size = batch_size, collate_fn = val_data.collate_fn, pin_memory=True)# TODO: Define the val loader. Remember to pass in a parameter (function) for the collate_fn argument 
test_loader = DataLoader(test_data, shuffle= False, batch_size = batch_size, collate_fn = test_data.collate_fn, pin_memory=True)# TODO: Define the test loader. Remember to pass in a parameter (function) for the collate_fn argument 



print("Batch size: ", batch_size)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  128
Train dataset samples = 28539, batches = 223
Val dataset samples = 2703, batches = 22
Test dataset samples = 2620, batches = 21


In [12]:
# test code for checking shapes
for data in val_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, len(ly))
    print(y[0]) # desired 
    break

torch.Size([128, 3204, 13]) torch.Size([128, 470]) torch.Size([128]) 128
tensor([ 0, 13, 25, 19,  5, 12,  6, 28,  9, 28,  1, 12, 23,  1, 25, 19, 28, 11,
        14,  5, 23, 28, 19,  8,  5, 28,  8,  1,  4, 28,  9, 20, 28,  9, 14, 28,
         8,  5, 18, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,

# Model (TODO)

Details about the experiments you ran, architectures you tried, number of epochs you trained for, hyperparameters you used (and specify which combination resulted in the best score), as well as your data loading scheme:

(1) The architectures I used is a LAS(attention) model includes two main modules: encoder and decoder. The encoder is based on 1 layer of bidirectional lstm(bottom) followed by 3 layers of pyramidal lstm and followed by value network and key network which are used for the attention mechanism, combined with the query obtained in the decoder. The decoder consists of two lstm cells and use the attention mechanism to output context and attentions. 

(2) Since as we told in recitation, tuning hyperparameters for optimizer and scheduler will not improve a lot, I foucs more on tuning architecture of my model:

Hyper-parameters for model structure:
{
    'encoder_hidden_dim':256,
    'decoder_hidden_dim':512,
    'embed_dim':256,
    'key_value_size': 128
}

PLSTM Layer(for encoder):

layers = 1,2(preferred)
bidirection = True
dropout = 0.5
hidden_size = encoder_hidden_dim

Lockeddropout: True(preferred), False
Drop_out ratio: 0.2, 0.3(preferred), 0.5


Other parameters: 

Number of epochs for baseline model(30, 50(preferred), 70) with tf droping from 1. I tried tf fixed to be 1 for first 50 epochs but it does not improve performance in general but also causes slow convergence. Thus, I let tf froping from 1 slowly, and actually it stops improving around 50 epochs. But overfitting is not very sustantial after 50 epochs. After trained this model, I saved this model, then added time masking and frequency masking and keep training by increasing tf to 0.9 and gradually drops to 0.6 for sufficient epochs.

batch size: 128 -> default


Optimizer:  optim.Adam(model.parameters(),lr=0.001)-->default

Scheduler (on lev distance):  optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2, threshold=1e-2)

Criterion: nn.CrossEntropyLoss(reduction='none') ->defualt


Data Augmentation: None, time masking and frequency masking(preferred)

data augmentation parameters: freq_mask_param = 3, time_mask_param = 150(preferred), 200

Data loading scheme: We used LibriSample and LibriSampletTest to parse the training and validation dataset and pad input sequence since input may not have same length, and we need to pack padded sequence when passing to lstm cell and unpack sequence during maksing(and we need to permute dimensions) as neccessary. In my setting, I always set batch_first = True.
The output of the decoder is in character probability distribution(not softmax) over indexes and we need to use greedy search and transform_index_to_letter to output the final result. This string will be used for calculating the levenstein distance between the ground truth. The final output submission to kaggle is recorded in a csv file called 'report.csv', which first columns is index/id, the second column is the predicted string that consists of strings. This is done by converting the greedy search's results(indexes) to LETTER_LIST strings and we recorded them in the dataframe(report.csv) corresponding to each id. 


Please Note: I used several trained models to keep training and get the optimal values, so this notebook may not includes all the models. But the main idea is to increase tf a little bit and keep training until converging to best performance one.

And when using locked_dropout, training from the "very beginning" may see convergence only after epoch 17 or later, please be patient on this. The distance will converge/shrink quite a lot after that.


In [13]:
# locked dropout module
from torch.autograd import Variable

class LockedDropout(nn.Module):
    def __init__(self, p = 0.3):
        self.p = p
        super().__init__()
        
    def forward(self,x):
        # x dim: (B,T,C)
        if self.get_parameter == 0:
            return x
        out, x_lens = pad_packed_sequence(x, batch_first=True)
        mask = out.data.new(out.shape[0],1,out.shape[2])
        mask = mask.bernoulli_(1-self.p)
        mask = Variable(mask, requires_grad=False)/(1-self.p)
        mask = mask.expand_as(out)
        masked_x = mask*out
        return pack_padded_sequence(masked_x, lengths=x_lens.cpu(), batch_first=True, enforce_sorted=False)

In [14]:
class pBLSTM(nn.Module):
    '''
    Pyramidal BiLSTM
    Read paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed
    2. Truncate the input length dimension by concatenating feature dimension
        (i) How should  you deal with odd/even length input? 
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''
    def __init__(self, input_dim, hidden_dim):
        super(pBLSTM, self).__init__()
        # the base/bottom lstm layer
        self.blstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=2, bidirectional=True, batch_first=True, dropout=0.5)


    def forward(self, x):
      # Pad your input if it is packed
      x_unpacked, x_lens = pad_packed_sequence(x, batch_first=True)
      # x_unpacked: shape[B, T, dim]
      # Truncate the input length dimension by concatenating feature dimension
      x_lens = x_lens.cuda()
      # chop off last element if input is odd
      chopped_length = torch.div(x_unpacked.shape[1],2, rounding_mode='floor')*2
      x_unpacked = x_unpacked[:,:chopped_length,:]
      # deal with input length array (x_lens) after truncating the input?
      # reshape to (B, T/2, dim*2)
      new_seq_len = torch.div(x_unpacked.shape[1],2, rounding_mode='floor')
      x_reshaped = x_unpacked.reshape(x_unpacked.shape[0], new_seq_len, x_unpacked.shape[2] * 2)
      x_lens = torch.div(x_lens, 2, rounding_mode='floor')

      # need to repack the seqences to serve as input to lstm layer(similar to HW3 part2)
      x_packed = pack_padded_sequence(x_reshaped, lengths=x_lens.cpu(), batch_first=True, enforce_sorted=False)

      # pass into the lstm layer
      out, hiddens = self.blstm(x_packed)
      return out
        

In [15]:
class Encoder(nn.Module):
    '''
    Encoder takes the utterances as inputs and returns the key, value and unpacked_x_len.

    '''
    def __init__(self, input_dim, encoder_hidden_dim, key_value_size=128):
        super(Encoder, self).__init__()
        # The first LSTM layer at the bottom
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size= encoder_hidden_dim, num_layers=1, bidirectional=True, batch_first=True)

        # Define the blocks of pBLSTMs
        # Dimensions should be chosen carefully
        # Hint: Bidirectionality, truncation-> x 4
        # Optional dropout
        """Design: 1 bottom BLSTM, 3 pBLSTMs on top, Reducing input length by factor of 8
        """
        self.pBLSTMs = nn.Sequential(pBLSTM(encoder_hidden_dim*4, encoder_hidden_dim),
            LockedDropout(p = 0.3),
            pBLSTM(encoder_hidden_dim*4, encoder_hidden_dim),
            LockedDropout(p = 0.3),
            pBLSTM(encoder_hidden_dim*4, encoder_hidden_dim)
        )
         
        # The linear transformations for producing Key and Value for attention
        # Hint: Dimensions when bidirectional lstm? --> x 2
        self.key_network = nn.Linear(encoder_hidden_dim*2, key_value_size)
        self.value_network = nn.Linear(encoder_hidden_dim*2, key_value_size)

    def forward(self, x, x_len):
        """
        1. Pack your input and pass it through the first LSTM layer (no truncation)
        2. Pass it through the pyramidal LSTM layer
        3. Pad your input back to (B, T, *) or (T, B, *) shape
        4. Output Key, Value, and truncated input lens

        Key and value could be
            (i) Concatenated hidden vectors from all time steps (key == value).
            (ii) Linear projections of the output from the last pBLSTM network.
                If you choose this way, you can use the final output of
                your pBLSTM network.
        """
        # Pack your input and pass it through the first LSTM layer (no truncation)
        x_packed = pack_padded_sequence(x, lengths= x_len.cpu(), batch_first=True, enforce_sorted=False)
        out,_ = self.lstm(x_packed)
    

        # Pass it through the pyramidal LSTM layer
        out = self.pBLSTMs(out)

        # Pad your input back to (B, T, *) shape
        linear_in, truncated_lens = pad_packed_sequence(out, batch_first=True)
        keys = self.key_network(linear_in)
        value = self.value_network(linear_in)
        # Output Key, Value, and truncated input lens(1/8 of original input length )
        return keys, value, truncated_lens

      

In [16]:
def plot_attention(attention):
    # utility function for debugging
    plt.clf()
    sns.heatmap(attention, cmap='GnBu')
    plt.show()

class Attention(nn.Module):
    '''
    Attention is calculated using key and value from encoder and query from decoder.
    Here are different ways to compute attention and context:
    1. Dot-product attention
        energy = bmm(key, query) 
        # Optional: Scaled dot-product by normalizing with sqrt key dimension
        # Check "attention is all you need" Section 3.2.1
    * 1st way is what most TAs are comfortable with, but if you want to explore...
    2. Cosine attention
        energy = cosine(query, key) # almost the same as dot-product xD 
    3. Bi-linear attention
        W = Linear transformation (learnable parameter): d_k -> d_q
        energy = bmm(key @ W, query)
    4. Multi-layer perceptron
        # Check "Neural Machine Translation and Sequence-to-sequence Models: A Tutorial" Section 8.4
    
    After obtaining unnormalized attention weights (energy), compute and return attention and context, i.e.,
    energy = mask(energy) # mask out padded elements with big negative number (e.g. -1e9)
    attention = softmax(energy)
    context = bmm(attention, value)

    5. Multi-Head Attention
        # Check "attention is all you need" Section 3.2.2
        h = Number of heads
        W_Q, W_K, W_V: Weight matrix for Q, K, V (h of them in total)
        W_O: d_v -> d_v

        Reshape K: (B, T, d_k)
        to (B, T, h, d_k // h) and transpose to (B, h, T, d_k // h)
        Reshape V: (B, T, d_v)
        to (B, T, h, d_v // h) and transpose to (B, h, T, d_v // h)
        Reshape Q: (B, d_q)
        to (B, h, d_q // h)

        energy = Q @ K^T
        energy = mask(energy)
        attention = softmax(energy)
        multi_head = attention @ V
        multi_head = multi_head reshaped to (B, d_v)
        context = multi_head @ W_O
    '''
    def __init__(self):
        super(Attention, self).__init__()
        # Optional: dropout


    def forward(self, query, key, value, mask):
        """
        input:
            key: (batch_size, seq_len, d_k)
            value: (batch_size, seq_len, d_v)
            query: (batch_size, d_q) --> output from decoder, one for each sequence
        * Hint: d_k == d_v == d_q is often true if you use linear projections
        return:
            context: (batch_size, key_val_dim)
        
        """
        # from documentation: torch.bmm(input, mat2, *, out=None) → Tensor
        # input and mat2 must be 3-D tensors each containing the same number of matrices.
        # so we need to first "unsqueeze query to 3D"
        query_unsqueezed = torch.unsqueeze(query,2)
        # batch, seq_len, d_k) * (batch, d_q, 1) = (batch, seq, 1) since d_k == d_v == d_q
        # we squeezed (N, seq_len)
        energy_unsqueezed = torch.bmm(key, query_unsqueezed) #(batch, seq, 1)
        energy = torch.squeeze(energy_unsqueezed, 2) # (batch, seq)

        mask = mask.cuda()
        energy.masked_fill_(mask, -float("inf")) # need use a boolean mask
        # scaled dot product for attention--optional
        energy = energy/np.sqrt(key.shape[2])
        # compute attention and context
        attention = nn.functional.softmax(energy, dim=1) # (batch, seq)
        # to bmm, we need to unsqueeze attention to 3D
        context = torch.bmm(attention.unsqueeze(1), value) #--> (batch, 1,seq_len) * (batch_size, seq_len, d_v) = (batch, 1, d_v)
        # squeeze out to (batch, d_v)
        context = torch.squeeze(context,1)
        # return context,attention
        return context,attention
        # we return attention weights for plotting (for debugging)

In [17]:
class Decoder(nn.Module):
    '''
    As mentioned in a previous recitation, each forward call of decoder deals with just one time step.
    Thus we use LSTMCell instead of LSTM here.
    The output from the last LSTMCell can be used as a query for calculating attention.
    Methods like Gumble noise and teacher forcing can also be incorporated for improving the performance.
    '''
    def __init__(self, vocab_size, decoder_hidden_dim, embed_dim, key_value_size=128):
        super(Decoder, self).__init__()
        # Hint: Be careful with the padding_idx
        padding_idx= letter2index['<eos>']
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx= padding_idx)

        # The number of cells is defined based on the paper--> 2 layers of RNN
        self.lstm1 = nn.LSTMCell(input_size = embed_dim + key_value_size, hidden_size = decoder_hidden_dim)
        self.lstm2 = nn.LSTMCell(input_size = decoder_hidden_dim, hidden_size = key_value_size)

        self.attention = Attention()     
        self.vocab_size = vocab_size
      
        self.character_prob = nn.Linear(2*key_value_size, vocab_size) #: d_v -> vocab_size
        self.key_value_size = key_value_size
        
        # Weight tying
        self.character_prob.weight = self.embedding.weight

    def forward(self, key, value, encoder_len, y=None, teacher_forcing_rate = 1, Gumbel_noise=False, mode='train'):
        '''
        Args:
            key :(B, T, d_k) - Output of the Encoder (possibly from the Key projection layer)
            value: (B, T, d_v) - Output of the Encoder (possibly from the Value projection layer)
            y: (B, text_len) - Batch input of text with text_length
            mode: Train or eval mode for teacher forcing
        Return:
            predictions: the character perdiction probability 
        '''
        

        B, key_seq_max_len, key_value_size = key.shape

        if mode == 'train':
            max_len =  y.shape[1]
            char_embeddings = self.embedding(y)
        else:
            max_len = 600

        # TODO: Create the attention mask here (outside the for loop rather than inside) to aviod repetition
        # If you have the numbers from 0-L and you check which are less than the sequence length, 
        # then it is real, and we don't mask it with negative number(mask = False )
        # if is larger or eqaul to encoder_len, then it is padded(not real) and we want to mask it(mask = True)
        mask = torch.arange(key_seq_max_len).unsqueeze(0) >= encoder_len.unsqueeze(1)
        mask = mask.type(torch.BoolTensor)
        predictions = []
        # This is the first input to the decoder
        # What should the fill_value be?
        prediction = torch.full((B,1), fill_value= 0 , device=device)
        # The length of hidden_states vector should depend on the number of LSTM Cells defined in init
        # The paper uses 2
        hidden_states = [None, None] 
        
        # TODO: Initialize the context
        context = value[:, 0, :]

        attention_plot = [] # this is for debugging

        for i in range(max_len):
            if mode == 'train':
                # TODO: only train, Implement Teacher Forcing
                using_teacher_forcing = np.random.random() < teacher_forcing_rate
                if using_teacher_forcing:
                    if i == 0:
                        # This is the first time step
                        # Hint: How did you initialize "prediction" variable above?--> fill with index of sos(start) for all batches
                        start = torch.zeros(B,dtype=torch.long).fill_(letter2index['<sos>'])
                        start = torch.LongTensor(start).cuda() #move to gpu
                        char_embed = self.embedding(start)
                    else:
                        # Otherwise, feed the label of the **previous** time step
                        char_embed = char_embeddings[:, i-1, :]
                else:
                    if i == 0:
                        # This is the first time step
                        # Hint: How did you initialize "prediction" variable above?--> fill with index of sos(start) for all batches
                        start = torch.zeros(B,dtype=torch.long).fill_(letter2index['<sos>'])
                        start = torch.LongTensor(start).cuda() #move to gpu
                        char_embed = self.embedding(start)
                    else:
                        previous_pred = prediction.argmax(dim=-1)
                        char_embed = self.embedding(previous_pred)#embedding of the previous prediction
                    
            else:
                # if the start
                if i == 0:
                    start = torch.zeros(B,dtype=torch.long).fill_(letter2index['<sos>'])
                    start = torch.LongTensor(start).cuda() #move to gpu
                    char_embed = self.embedding(start)  

                else:
                    previous_pred = prediction.argmax(dim=-1)
                    char_embed = self.embedding(previous_pred)# embedding of the previous prediction

            # what vectors should be concatenated as a context? --> char_embed, context, 
            y_context = torch.cat([char_embed, context], dim=1)
            # context and hidden states of lstm 1 from the previous time step should be fed
            hidden_states[0] = self.lstm1(y_context, hidden_states[0])

            # hidden states of lstm1 and hidden states of lstm2 from the previous time step should be fed
            out_prev_layer = hidden_states[0][0]
            hidden_states[1] = self.lstm2(out_prev_layer, hidden_states[1])

            # What then is the query?
            query = hidden_states[1][0]
            
            # Compute attention from the output of the second LSTM Cell
            context, attention = self.attention(query, key, value, mask)
            # We store the first attention of this batch for debugging
            attention_plot.append(attention[0].detach().cpu())
            
            # What should be concatenated as the output context?
            output_context = torch.cat([query,context], dim=1)
            prediction = self.character_prob(output_context)
            # store predictions
            prediction_unsqueezed = torch.unsqueeze(prediction,1) # this is for concatenate for each batch
            # for each bacth, we keep concatenate prediction over vocab_size for each step in range(0,max_len)
            predictions.append(prediction_unsqueezed)
        
        # Concatenate the attention and predictions to return
        attentions = torch.stack(attention_plot, dim=0)
        #plot_attention(attentions)
        predictions = torch.cat(predictions, dim=1)
        return predictions, attentions

In [18]:
class Seq2Seq(nn.Module):
    '''
    We train an end-to-end sequence to sequence model comprising of Encoder and Decoder.
    This is simply a wrapper "model" for your encoder and decoder.
    '''
    def __init__(self, input_dim, vocab_size, encoder_hidden_dim, decoder_hidden_dim, embed_dim, key_value_size=128):
        super(Seq2Seq,self).__init__()
        self.encoder = Encoder(input_dim, encoder_hidden_dim)
        self.decoder = Decoder(vocab_size, decoder_hidden_dim, embed_dim, key_value_size)

    def forward(self, x, x_len, y=None, teacher_forcing_rate = 1, Gumbel_noise=False, mode='train'):
        key, value, encoder_len = self.encoder(x, x_len)
      
        predictions, attentions = self.decoder(key, value, encoder_len, y=y, teacher_forcing_rate = teacher_forcing_rate,  Gumbel_noise=False, mode=mode)
        return predictions

In [19]:
# define some hyper-parameters
hyper_parameters = {
    'encoder_hidden_dim':256,
    'decoder_hidden_dim':512,
    'embed_dim':256,
    'key_value_size': 128
    }

In [20]:
model = Seq2Seq(input_dim = 13, vocab_size = len(LETTER_LIST), 
                encoder_hidden_dim = hyper_parameters['encoder_hidden_dim'], decoder_hidden_dim = hyper_parameters['decoder_hidden_dim'],
                embed_dim = hyper_parameters['embed_dim'], key_value_size = hyper_parameters['key_value_size']
                )

model = model.to(device)
print(model)

Seq2Seq(
  (encoder): Encoder(
    (lstm): LSTM(13, 256, batch_first=True, bidirectional=True)
    (pBLSTMs): Sequential(
      (0): pBLSTM(
        (blstm): LSTM(1024, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
      )
      (1): LockedDropout()
      (2): pBLSTM(
        (blstm): LSTM(1024, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
      )
      (3): LockedDropout()
      (4): pBLSTM(
        (blstm): LSTM(1024, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
      )
    )
    (key_network): Linear(in_features=512, out_features=128, bias=True)
    (value_network): Linear(in_features=512, out_features=128, bias=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(30, 256, padding_idx=29)
    (lstm1): LSTMCell(384, 512)
    (lstm2): LSTMCell(512, 128)
    (attention): Attention()
    (character_prob): Linear(in_features=256, out_features=30, bias=True)
  )
)


# Training

In [21]:
optimizer = optim.Adam(model.parameters(),lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2, threshold=1e-2)
criterion = nn.CrossEntropyLoss(reduction='none')

In [22]:
# define some useful, helper functions
def generate_mask(target_len):
    target_len = target_len.clone().detach()
    max_len = torch.max(target_len)

    mask = (torch.arange(0, max_len).repeat(target_len.shape[0], 1).to(device) < \
                target_len.unsqueeze(1).expand(target_len.shape[0], max_len)).int() #transoform to numerical
    
    
    return mask





In [23]:
def calculate_levenshtein(text_prediction, text_target):
    # for a single batch
    total_distance = 0
    for pred, target in zip(text_prediction, text_target):
        dist = lev.distance(pred, target)
        total_distance += dist
    return total_distance

In [24]:
def val(model, val_loader, mode = "test"):
    dist = 0
    avg_loss = 0
    num_of_seq = 0
    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
    for i, (x,y,x_len,y_len) in enumerate(val_loader): # for a single batch
        x,y,x_len,y_len = x.to(device),y.to(device),x_len.to(device),y_len.to(device)
        with torch.no_grad():
            predictions = model(x, x_len, y, mode="test") # no need for teaching, teaching rate only for training 
        
           
        text_prediction = transform_index_to_letter(predictions.argmax(-1).detach().cpu().numpy()) #greedy
        text_target = transform_index_to_letter(y.detach().cpu().numpy())
        
        #mask = generate_mask(y_len).to(device)
            
        # 4) Make sure you have the correct shape of predictions when putting into criterion
        """print("shape of predictions: ",predictions.shape)
        print("shape of y is: ", y.shape)
        print("pred reshaped: ", predictions.view(-1, predictions.shape[2]).shape)
        print("y reshaped: ", y.view(-1).shape)
        loss = criterion(predictions.view(-1, predictions.shape[2]), y.view(-1))
        # Use the mask you defined above to compute the average loss
        #masked_loss = torch.sum(loss * mask.view(-1)) / torch.sum(mask)
        masked_loss = torch.div(torch.sum(loss * mask.view(-1)), torch.sum(mask), rounding_mode='floor')
        avg_loss += masked_loss.item()"""
        dist += calculate_levenshtein(text_prediction, text_target)
        num_of_seq += len(text_prediction) # add number of sequences for each batch

        #batch_bar.set_postfix(loss="{:.04f}".format(masked_loss))
        batch_bar.update()
    batch_bar.close()
    #average_loss = avg_loss/num_of_seq
    average_distance = dist/num_of_seq
    print("Average Levestein distance: {:.04f}".format(average_distance))
    return average_distance

In [25]:
torch.cuda.empty_cache()

In [27]:
model = Seq2Seq(input_dim = 13, vocab_size = len(LETTER_LIST), 
                encoder_hidden_dim = hyper_parameters['encoder_hidden_dim'], decoder_hidden_dim = hyper_parameters['decoder_hidden_dim'],
                embed_dim = hyper_parameters['embed_dim'], key_value_size = hyper_parameters['key_value_size']
                )

model.to(device)

Seq2Seq(
  (encoder): Encoder(
    (lstm): LSTM(13, 256, batch_first=True, bidirectional=True)
    (pBLSTMs): Sequential(
      (0): pBLSTM(
        (blstm): LSTM(1024, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
      )
      (1): LockedDropout()
      (2): pBLSTM(
        (blstm): LSTM(1024, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
      )
      (3): LockedDropout()
      (4): pBLSTM(
        (blstm): LSTM(1024, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
      )
    )
    (key_network): Linear(in_features=512, out_features=128, bias=True)
    (value_network): Linear(in_features=512, out_features=128, bias=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(30, 256, padding_idx=29)
    (lstm1): LSTMCell(384, 512)
    (lstm2): LSTMCell(512, 128)
    (attention): Attention()
    (character_prob): Linear(in_features=256, out_features=30, bias=True)
  )
)

In [ ]:
# TODO: Define your model and put it on the device here
# main train process-->keep this!
from torch.nn.utils.clip_grad import clip_grad_norm_
n_epochs =  100
tf_upperbound = 1
tf_lowerbound = 0.6
optimizer = optim.Adam(model.parameters(), lr = 0.001)
# Make sure you understand the implication of setting reduction = 'none'
criterion = nn.CrossEntropyLoss(reduction='none')
mode = 'train'


# first train for teacher_forcing_rate = 1 until convergence(distance--> below 50)
for epoch in range(n_epochs):
    #train(model, train_loader, criterion, optimizer, mode)
    model.train()
    running_loss = 0
    teacher_forcing_rate = tf_upperbound - (tf_upperbound-tf_lowerbound)*(epoch/n_epochs)
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    # 0) Iterate through your data loader
    for i, (x,y,x_len,y_len) in enumerate(train_loader):
        
        # 1) Send the inputs to the device
        x,y,x_len,y_len = x.to(device),y.to(device),x_len.to(device),y_len.to(device)
        #  data augmentation
        x = train_audio_transforms(x.permute(0,2,1)).permute(0,2,1)
        
        optimizer.zero_grad()

        # 2) Pass your inputs, and length of speech into the model.
        
        predictions= model(x, x_len, y, teacher_forcing_rate = teacher_forcing_rate, Gumbel_noise=False, mode=mode)
        
        
        # 3) Generate a mask based on target length. This is to mark padded elements
        # so that we can exclude them from computing loss.
        # Ensure that the mask is on the device and is the correct shape.
        mask = generate_mask(y_len).to(device)
        """print("shape of predictions: ",predictions.shape)
        print("shape of y is: ", y.shape)
        print("pred reshaped: ", predictions.view(-1, predictions.shape[2]).shape)
        print("y reshaped: ", y.view(-1).shape)"""
            
        # 4) Make sure you have the correct shape of predictions when putting into criterion
        loss = criterion(predictions.view(-1, predictions.shape[2]), y.view(-1))
        # Use the mask you defined above to compute the average loss
        
        masked_loss = torch.sum(loss * mask.view(-1)) / torch.sum(mask) #normalize
        batch_bar.set_postfix(loss="{:.04f}".format(float(masked_loss)))
        # 5) backprop
        masked_loss.backward()
        # Optional: Gradient clipping
        clip_grad_norm_(model.parameters(), 2)
        optimizer.step()
        batch_bar.update()
        # need to do this:
        torch.cuda.empty_cache()
        # When computing Levenshtein distance, make sure you truncate prediction/target
  
    batch_bar.close()
    print("Epoch {}/{}:".format(epoch + 1, n_epochs))
    average_distance = val(model, val_loader, mode = "test")
    scheduler.step(average_distance)
    # Optional: plot your attention for debugging
    # plot_attention(attentions)

    

Epoch 1/100:


Average Levestein distance: 531.8076


Epoch 2/100:


Average Levestein distance: 533.8827


Epoch 3/100:


Average Levestein distance: 536.0485


Train:  17%|█▋        | 37/223 [03:36<17:37,  5.69s/it, loss=1.2955]

In [ ]:
# save model and keep train with lower teacher forcing rate
torch.save(model.state_dict(), 'LockedDropout model, after train'+ str(epoch))


In [ ]:
model = Seq2Seq(input_dim = 13, vocab_size = len(LETTER_LIST), 
                encoder_hidden_dim = hyper_parameters['encoder_hidden_dim'], decoder_hidden_dim = hyper_parameters['decoder_hidden_dim'],
                embed_dim = hyper_parameters['embed_dim'], key_value_size = hyper_parameters['key_value_size']
                )
PATH = '/content/LockedDropout model, after train28'  # this needs to be fixed until convergence
model.load_state_dict(torch.load(PATH))
model.to(device)

In [ ]:
torch.cuda.empty_cache()

In [ ]:

# for retrain
from torch.nn.utils.clip_grad import clip_grad_norm_
n_epochs =  60
tf_upperbound = 0.9
tf_lowerbound = 0.6
optimizer = optim.Adam(model.parameters(), lr = 0.001)
# Make sure you understand the implication of setting reduction = 'none'
criterion = nn.CrossEntropyLoss(reduction='none')
mode = 'train'


# first train for teacher_forcing_rate = 1 until convergence(distance--> below 50)
for epoch in range(n_epochs):
    #train(model, train_loader, criterion, optimizer, mode)
    model.train()
    running_loss = 0
    teacher_forcing_rate = tf_upperbound - (tf_upperbound-tf_lowerbound)*(epoch/n_epochs)
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    # 0) Iterate through your data loader
    for i, (x,y,x_len,y_len) in enumerate(train_loader):
        
        # 1) Send the inputs to the device
        x,y,x_len,y_len = x.to(device),y.to(device),x_len.to(device),y_len.to(device)
        #  data augmentation
        x = train_audio_transforms(x.permute(0,2,1)).permute(0,2,1)
        #x = train_audio_transforms(x.permute(2,0,1)).permute(1,2,0)
        optimizer.zero_grad()

        # 2) Pass your inputs, and length of speech into the model.
        
        predictions= model(x, x_len, y, teacher_forcing_rate = teacher_forcing_rate, Gumbel_noise=False, mode=mode)
        
        
        # 3) Generate a mask based on target length. This is to mark padded elements
        # so that we can exclude them from computing loss.
        # Ensure that the mask is on the device and is the correct shape.
        mask = generate_mask(y_len).to(device)
        """print("shape of predictions: ",predictions.shape)
        print("shape of y is: ", y.shape)
        print("pred reshaped: ", predictions.view(-1, predictions.shape[2]).shape)
        print("y reshaped: ", y.view(-1).shape)"""
            
        # 4) Make sure you have the correct shape of predictions when putting into criterion
        loss = criterion(predictions.view(-1, predictions.shape[2]), y.view(-1))
        # Use the mask you defined above to compute the average loss
        #masked_loss = torch.div(torch.sum(loss * mask.view(-1)), torch.sum(mask), rounding_mode='floor')
        masked_loss = torch.sum(loss * mask.view(-1)) / torch.sum(mask) #normalize
        batch_bar.set_postfix(loss="{:.04f}".format(float(masked_loss)))
        # 5) backprop
        masked_loss.backward()
        # Optional: Gradient clipping
        clip_grad_norm_(model.parameters(), 2)
        optimizer.step()
        batch_bar.update()
        # need to do this:
        torch.cuda.empty_cache()
        # When computing Levenshtein distance, make sure you truncate prediction/target
  
    batch_bar.close()
    print("Epoch {}/{}:".format(epoch + 1, n_epochs))
    average_distance = val(model, val_loader, mode = "test")
    scheduler.step(average_distance)
    # Optional: plot your attention for debugging
    # plot_attention(attentions)

    

In [ ]:
prediction = []
model.eval()
for batch_idx, (x, x_len) in enumerate(test_loader):
    x, x_len = x.to(device), x_len.to(device)
    predictions = model(x, x_len, y, mode="test")
    text_prediction = transform_index_to_letter(predictions.argmax(-1).detach().cpu().numpy()) #greedy
    prediction.extend(text_prediction)
    
ids = np.arange(0, len(prediction))
prediction = np.array(prediction)
df = pd.DataFrame({"id" : ids, "predictions" : prediction})
df.to_csv('/content/report.csv', index=False)

In [ ]:
'''
Debugging suggestions from Eason, a TA from previous semesters:

(1) Decrease your batch_size to 2 and print out the value and shape of all intermediate variables to check if they satisfy the expectation
(2) Be super careful about the LR, don't make it too high. Too large LR would lead to divergence and your attention plot will never make sense
(3) Make sure you have correctly handled the situation for time_step = 0 when teacher forcing

(1) is super important and is the most efficient way for debugging. 
'''
'''
Tips for passing A from B (from easy to hard):
** You need to implement all of these yourself without utilizing any library **
(1) Increase model capacity. E.g. increase num_layer of lstm
(2) LR and Teacher Forcing are also very important, you can tune them or their scheduler as well. Do NOT change lr or tf during the warm-up stage!
(3) Weight tying--> done
(4) Locked Dropout - insert between the plstm layers
(5) Pre-training decoder or train an LM to help make predictions
(5) Pre-training decoder to speed up the convergence: 
    disable your encoder and only train the decoder like train a language model
(6) Better weight initialization technique
(7) Batch Norm between plstm. You definitely can try other positions as well --》 not helpful
(8) Data Augmentation. Time-masking, frequency masking -》 done
(9) Weight smoothing (avg the last few epoch's weight)
(10) You can try CNN + Maxpooling (Avg). Some students replace the entire plstm blocks with it and some just combine them together.
(11) Beam Search
'''